In [74]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn import metrics
import xgboost as xgb
import tsfresh as ts
import matplotlib.pyplot as plt
import seaborn as sns

In [77]:
df = pd.read_csv('daily-temperatures.csv')
df.head()

,Date,Temp
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8


Extract year, month and day_of_month

In [58]:
df.Date = pd.to_datetime(df.Date)  # no unless we need to set a specifc format, otherwise it will use the default
df['year'] = df.Date.dt.year
df['month'] = df.Date.dt.month
df['day_of_month'] = df.Date.dt.day
df['quater'] = df.Date.dt.quarter

df_pred = df.copy()
# ok this works do you want to try the shift features?

Get lags features → Pandas shift()
    a.The temperature of day before, and 2 days ago

In [ ]:
df['temp_day_before'] = df.Temp.shift(1)
df['temp_2_days_before'] = df.Temp.shift(2)
df.head()

,Date,Temp,year,month,day_of_month,temp_day_before,temp_2_days_before
0,1981-01-01,20.7,1981,1,1,NaN,NaN
1,1981-01-02,17.9,1981,1,2,20.7,NaN
2,1981-01-03,18.8,1981,1,3,17.9,20.7
3,1981-01-04,14.6,1981,1,4,18.8,17.9
4,1981-01-05,15.8,1981,1,5,14.6,18.8


Rolling Window Statistics → Pandas rolling()a.Mean, min & max of 7 previous days

In [ ]:
df['rolling_temp_7'] = df.Temp.rolling(7).mean()
df['rolling_temp_7_min'] = df.Temp.rolling(7).min()
df['rolling_temp_7_max'] = df.Temp.rolling(7).max()
#df['rolling_temp_180'] = df.Temp.rolling(180).mean()
df.head(10)

,Date,Temp,year,month,day_of_month,id,date_num,rolling_temp_7,rolling_temp_7_min,rolling_temp_7_max
0,1981-01-01,20.7,1981,1,1,0,1,NaN,NaN,NaN
1,1981-01-02,17.9,1981,1,2,1,2,NaN,NaN,NaN
2,1981-01-03,18.8,1981,1,3,2,3,NaN,NaN,NaN
3,1981-01-04,14.6,1981,1,4,3,4,NaN,NaN,NaN
4,1981-01-05,15.8,1981,1,5,4,5,NaN,NaN,NaN
5,1981-01-06,15.8,1981,1,6,5,6,NaN,NaN,NaN
6,1981-01-07,15.8,1981,1,7,6,7,17.057143,14.6,20.7
7,1981-01-08,17.4,1981,1,8,7,8,16.585714,14.6,18.8
8,1981-01-09,21.8,1981,1,9,8,9,17.142857,14.6,21.8
9,1981-01-10,20.0,1981,1,10,9,10,17.314286,14.6,21.8


In [ ]:
df['id'] = df.index
df['date_num'] = df.Date.dt.day_of_year
#df = df.fillna(-9999)
df = df.dropna()

x = df.drop(['Temp','Date'], axis=1, inplace=False)
y = df.Temp

x.shape, y.shape

((3644, 8), (3644,))

Try new features. 

Mayb we can try the tsfresh here. 

Select the year 1990 as a validation set

In [ ]:
x_train = x[x.year != 1990]
y_train = df.Temp[df.year != 1990]

x_val = x[x.year == 1990]
y_val = df.Temp[df.year == 1990]
x_train.head()

x_train.shape, y_train.shape, x_val.shape, y_val.shape

((3279, 8), (3279,), (365, 8), (365,))

In [ ]:
pols = PolynomialFeatures(2)
x_train = pols.fit_transform(x_train)
x_val = pols.transform(x_val)
x_train

array([[1.00000000e+00, 1.98100000e+03, 1.00000000e+00, ...,
        2.52000000e+02, 2.94000000e+02, 3.43000000e+02],
       [1.00000000e+00, 1.98100000e+03, 1.00000000e+00, ...,
        3.92000000e+02, 4.48000000e+02, 5.12000000e+02],
       [1.00000000e+00, 1.98100000e+03, 1.00000000e+00, ...,
        5.76000000e+02, 6.48000000e+02, 7.29000000e+02],
       ...,
       [1.00000000e+00, 1.98900000e+03, 1.20000000e+01, ...,
        3.91006321e+09, 4.32465858e+08, 4.78321470e+07],
       [1.00000000e+00, 1.98900000e+03, 1.20000000e+01, ...,
        3.92322440e+09, 4.34984368e+08, 4.82285440e+07],
       [1.00000000e+00, 1.98900000e+03, 1.20000000e+01, ...,
        3.93639944e+09, 4.37510900e+08, 4.86271250e+07]])

Try some some ML models to predict 1990a.You can use the RMSE as the metric

In [ ]:
lr = LinearRegression()
lr.fit(x_train,y_train)
preds = lr.predict(x_val)
m_score = lr.score(x_val,y_val)
score = metrics.mean_squared_error(y_val,preds, squared=False) #RMSE
score, m_score

(2.0366467541097126, 0.7210744811108067)

In [ ]:
br = BayesianRidge()
br.fit(x_train,y_train)
br_preds = br.predict(x_val)
br_score = br.score(x_val,y_val)
br_rmse = metrics.mean_squared_error(y_val,br_preds, squared=False)

br_rmse, br_score

(2.0378465171838003, 0.7207457612630866)

In [ ]:
rf = RandomForestRegressor()
rf.fit(x_train,y_train)
rf_preds = rf.predict(x_val)
rf_rmse = metrics.mean_squared_error(y_val, rf_preds)
rf_score = rf.score(x_val,y_val)
rf_rmse, rf_score

(5.521744709589043, 0.6286929847140723)

2.292076352334736

In [ ]:
df_1989 = df[df.year == 1989]
y_89 = df_1989.Temp
last_rmse = metrics.mean_squared_error(y_val, y_89)
last_rmse

13.328054794520547

# No Data Leak Section

In [59]:
df_pred["day_year_before"] = df_pred.Temp.shift(365)
df_pred["Year_bef_roll_7"] = df_pred.day_year_before.rolling(7,win_type="bartlett").mean()
#df_pred["Year_bef_roll_7_min"] = df_pred.day_year_before.rolling(7).min()
#df_pred["Year_bef_roll_7_max"] = df_pred.day_year_before.rolling(7).max()
df_pred["Year_bef_roll_21"] = df_pred.day_year_before.rolling(21,win_type="bartlett").mean()
#df_pred["Year_bef_roll_31"] = df_pred.day_year_before.rolling(31,win_type="bartlett").mean()
#df_pred["Year_bef_roll_61"] = df_pred.day_year_before.rolling(61,win_type="bartlett").mean()
df_pred = df_pred.dropna()

### Average monthly tempature 

In [72]:
#Average Temp for each month. 
months = df_pred.groupby(by='month')['Temp'].agg("mean")

df_pred["month_ave"] = df_pred['month'].apply(lambda x: months[x])
df_pred.head()

,Date,Temp,year,month,day_of_month,quater,day_year_before,Year_bef_roll_7,Year_bef_roll_21,month_ave,qut_ave
385,1982-01-21,20.5,1982,1,21,1,18.2,20.800000,18.395,14.579923,14.789646
386,1982-01-22,19.3,1982,1,22,1,12.1,19.166667,18.594,14.579923,14.789646
387,1982-01-23,15.8,1982,1,23,1,14.4,17.244444,18.827,14.579923,14.789646
388,1982-01-24,17.0,1982,1,24,1,16.0,15.766667,19.058,14.579923,14.789646
389,1982-01-25,18.4,1982,1,25,1,16.5,14.777778,19.165,14.579923,14.789646


## Quater Average 

In [62]:
qut = df_pred.groupby(by='quater')["Temp"].agg('mean')
df_pred['qut_ave'] = df_pred['quater'].apply(lambda x: qut[x])
df_pred.head()

,Date,Temp,year,month,day_of_month,quater,day_year_before,Year_bef_roll_7,Year_bef_roll_21,month_ave,qut_ave
385,1982-01-21,20.5,1982,1,21,1,18.2,20.800000,18.395,14.579923,14.789646
386,1982-01-22,19.3,1982,1,22,1,12.1,19.166667,18.594,14.579923,14.789646
387,1982-01-23,15.8,1982,1,23,1,14.4,17.244444,18.827,14.579923,14.789646
388,1982-01-24,17.0,1982,1,24,1,16.0,15.766667,19.058,14.579923,14.789646
389,1982-01-25,18.4,1982,1,25,1,16.5,14.777778,19.165,14.579923,14.789646


In [73]:
xp = df_pred.drop(['Temp','Date'], axis=1, inplace=False)
yp = df_pred.Temp

In [64]:
xp_train = xp[xp.year != 1990]
yp_train = df_pred.Temp[df.year != 1990]

xp_val = xp[xp.year == 1990]
yp_val = df_pred.Temp[df.year == 1990]


xp_train.shape, yp_train.shape, xp_val.shape, yp_val.shape

((2900, 9), (2900,), (365, 9), (365,))

In [71]:
rf = RandomForestRegressor()
rf.fit(xp_train,yp_train)
p_preds = rf.predict(xp_val)
p_rmse = metrics.mean_squared_error(yp_val, p_preds, squared=False)
p_score = rf.score(xp_val,yp_val)
p_rmse, p_score

(2.690172339542275, 0.5133496536679356)

In [65]:
plr = LinearRegression()
plr.fit(xp_train,yp_train)
plr_preds = plr.predict(xp_val)
pm_score = plr.score(xp_val,yp_val)
plrscore = metrics.mean_squared_error(yp_val,plr_preds, squared=False) #RMSE
plrscore, pm_score


(2.56933439093271, 0.5560867675424013)

In [66]:
pbr = BayesianRidge()
pbr.fit(xp_train,yp_train)
pbr_preds = pbr.predict(xp_val)
pbr_score = pbr.score(xp_val,yp_val)
pbr_rmse = metrics.mean_squared_error(yp_val,pbr_preds, squared=False)

pbr_rmse, pbr_score

(2.566854177832996, 0.5569433847374056)

In [67]:
knn = KNeighborsRegressor()
knn.fit(xp_train,yp_train)
knn_preds = knn.predict(xp_val)
knn_score = knn.score(xp_val,yp_val)
knn_rmse = metrics.mean_squared_error(yp_val,knn_preds, squared=False)
knn_rmse, knn_score

(3.0485481835452855, 0.3750533302338017)

In [68]:
svr = SVR()
svr.fit(xp_train, yp_train)
svr_preds = svr.predict(xp_val)
svr_score = svr.score(xp_val, yp_val)
svr_rmse = metrics.mean_squared_error(yp_val, svr_preds, squared=False)
svr_rmse, svr_score

(3.8868924783540737, -0.015925555372812594)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2226da77-d448-4a0c-98fe-2f5119a7e9d8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>